In [67]:
import pandas as pd

In [65]:
LOAN_AMT = 50000
LOAN_TERM = 60 #in month
APR = 0.1510 #in percentage

In [ ]:
# APR = (Interest / Loan amount) / Number of days in loan term)) x 365 x 100
APR * N / 36500 = (I+F) / P

In [14]:
625/50000/(60*365)

5.707762557077626e-07

In [6]:
I = 50000*0.15/12
I

625.0

In [112]:
def calculate_interest(P,r):
    '''Calculate simple monthly interest payment
    '''
    
    return P*r/12
    
def calculate_snapshot_monthly(P, r, n):
    '''Calculate amortized monthly payment
    
    P (int): initial principal
    r (float): interest per period (e.g., APR/12)
    n (int): total number of payments
    '''
    r /= 12 #interest per month
    A = P / ( (1+r)**n-1 ) * (r*(1+r)**n)
    
    return A

def generate_amortization_table(P,r,n,additive=0):
    '''Generate complete amortization schedule
    
    P (int): initial principal
    r (float): interest per period (e.g., APR/12)
    n (int): total number of payments
    additive (float): additional monthly principal payment
    '''
    cols = ['Beginning Balance', 'Total Pmt', 'Interest', 'Principal', 'Ending Balance']
    df = pd.DataFrame(columns=cols)
    M = calculate_snapshot_monthly(P=P, r=r, n=n) #monthly total (this is constant)
    
    for i in range(n):
        I = calculate_interest(P=P, r=r)
        P_i = P #initial balance
        pr = M-I #principal
        P -= pr+additive #ending balance
        tmp = [P_i, M, I, pr, P]
        tmp = [round(j,2) for j in tmp]
        
        df = pd.concat([df, pd.DataFrame(tmp, index=cols).T])
        df = df.reset_index(drop=True)
        df.index += 1
    return df

In [115]:
generate_amortization_table(P=LOAN_AMT, r=APR, n=LOAN_TERM, additive=50)

,Beginning Balance,Total Pmt,Interest,Principal,Ending Balance
1,50000.0,1192.12,629.17,562.96,49387.04
2,49387.04,1192.12,621.45,570.67,48766.37
3,48766.37,1192.12,613.64,578.48,48137.9
4,48137.9,1192.12,605.74,586.39,47501.51
5,47501.51,1192.12,597.73,594.4,46857.11
6,46857.11,1192.12,589.62,602.5,46204.61
7,46204.61,1192.12,581.41,610.71,45543.89
8,45543.89,1192.12,573.09,619.03,44874.87
9,44874.87,1192.12,564.68,627.45,44197.42
10,44197.42,1192.12,556.15,635.97,43511.45


In [90]:
calculate_snapshot_monthly(P=LOAN_AMT, r=APR, n=LOAN_TERM)

1192.122677298203

In [62]:
calculate_interest(P=LOAN_AMT, r=APR)

625.0

In [64]:
generate_amortization_table(P=LOAN_AMT, r=APR, n=LOAN_TERM)

564.4965043179393

In [44]:
LOAN_AMT / ( (1+APR)**LOAN_TERM-1 ) * (APR*(1+APR)**LOAN_TERM)

1189.4965043179393

In [7]:
(50000-I)*0.15/12

617.1875